In [0]:
!pip install flair -q

In [2]:
!wget -c https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
!unzip -o drugsCom_raw.zip

--2020-03-08 23:30:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [3]:
import pandas as pd

drugs = pd.read_csv('/content/drugsComTrain_raw.tsv', sep='\t', header=0, nrows=20000)
print(drugs.shape, drugs.columns)


(20000, 7) Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount'],
      dtype='object')


In [4]:
from flair.data import Sentence, Label

sentences = []
for idx in drugs.index:
  data = drugs.iloc[idx]
  sentence = Sentence(data.review, labels=[Label(value=str(data.rating))], use_tokenizer=True)
  sentences.append(sentence)

In [5]:
from flair.embeddings import WordEmbeddings
import gensim

!wget -c https://ghc-flair.s3.amazonaws.com/wiki.model
!wget -c https://ghc-flair.s3.amazonaws.com/wiki.model.vectors.npy

custom_embedding =  WordEmbeddings('wiki.model')

n_loop = 1

--2020-03-08 23:31:06--  https://ghc-flair.s3.amazonaws.com/wiki.model
Resolving ghc-flair.s3.amazonaws.com (ghc-flair.s3.amazonaws.com)... 52.217.2.4
Connecting to ghc-flair.s3.amazonaws.com (ghc-flair.s3.amazonaws.com)|52.217.2.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140148778 (134M) [binary/octet-stream]
Saving to: ‘wiki.model’

wiki.model          100%[===================>] 133.66M  40.5MB/s    in 3.3s    

2020-03-08 23:31:10 (40.5 MB/s) - ‘wiki.model’ saved [140148778/140148778]

--2020-03-08 23:31:11--  https://ghc-flair.s3.amazonaws.com/wiki.model.vectors.npy
Resolving ghc-flair.s3.amazonaws.com (ghc-flair.s3.amazonaws.com)... 52.217.45.36
Connecting to ghc-flair.s3.amazonaws.com (ghc-flair.s3.amazonaws.com)|52.217.45.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3023244128 (2.8G) [binary/octet-stream]
Saving to: ‘wiki.model.vectors.npy’

wiki.model.vectors. 100%[===================>]   2.82G  44.6MB/s    in 65s

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
from flair.data import Sentence, Corpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models.text_regression_model import TextRegressor
from flair.trainers import ModelTrainer


target_set = drugs['rating'].values

results = []

## TEST
kfold_test = StratifiedKFold(n_splits=3, random_state=13)
for i, (train, test) in enumerate(kfold_test.split(target_set, target_set)):

  if i > n_loop:
    break;

  if i < n_loop:
    continue;


  print(i, len(train), len(test))
  corpus = Corpus([sentences[t] for t in train], [sentences[d] for d in test], [sentences[e] for e in test]) 

  label_dict = corpus.make_label_dictionary()
  
  word_embeddings = [custom_embedding]
  
  document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                       hidden_size=512,
                                                                       reproject_words=True,
                                                                       reproject_words_dimension=256,
                                                                       )
  model = TextRegressor(document_embeddings)

  trainer = ModelTrainer(model, corpus)

  result = trainer.train('train_'+str(i)+'/')

  print(str(i) + " - Test score: " + str(result["test_score"]))


1 13333 6667
2020-03-08 23:32:24,572 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
100%|██████████| 13333/13333 [00:00<00:00, 123086.29it/s]

2020-03-08 23:32:24,708 [b'9.0', b'8.0', b'5.0', b'2.0', b'1.0', b'10.0', b'4.0', b'3.0', b'7.0', b'6.0']


2020-03-08 23:32:33,694 Using REGRESSION - experimental
2020-03-08 23:32:33,695 ----------------------------------------------------------------------------------------------------
2020-03-08 23:32:33,696 Model: "TextRegressor(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('wiki.model')
    )
    (word_reprojection_map): Linear(in_features=300, out_features=256, bias=True)
    (rnn): GRU(256, 512, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Linear(in_features=512, out_features=1, bias=True)
  (loss_function): MSELoss()
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2020-03-08 23:32:33,697 ----------------------------------------------------------------------------------------------------
2020-03-08 23:32:33,698 Corpus: "Corpus: 13333 train + 6667 dev + 6667 test sentences"
2020-03-08 23:32:33,699 -----------------------------------------------------------------